In [ ]:
# (그래프에 한글이 깨질경우) 아래 코드 실행 > '런타임: 세션 다시시작' 실행
!pip install matplotlib -U
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## **라이브러리 다운로드 및 불러오기**

In [ ]:
# 라이브러리 불러오기: 데이터 핸들링
import pandas as pd
import numpy as np

# 라이브러리 불러오기: 연관성 분석
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **데이터**
  - 데이터명: 전자상거래 내, 카테고리별 고객 행동 데이터
  - 출처: 캐글 오픈 데이터(https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store)

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_02.csv')
print(df.shape)
df.tail()

(67501979, 11)


,Unnamed: 0,KTC_일시,이벤트 타입,상품ID,상품 카테고리ID,카테고리 코드,브랜드,가격,사용자ID,사용자 세션ID,kTC_일(Day)
67501974,67501974,2019-12-01 08:59:58,view,15700137,2053013559733912211,NaN,NaN,277.74,532714000,02b4131c-0112-4231-aafa-ceaa08e77c1b,1
67501975,67501975,2019-12-01 08:59:58,view,28719425,2053013565639492569,apparel.shoes,baden,62.81,545223467,734c5eef-0742-4f8b-9d22-48f75b0bc359,1
67501976,67501976,2019-12-01 08:59:59,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.03,557794415,6fecf566-ebb0-4e70-a243-cdc13ce044cb,1
67501977,67501977,2019-12-01 08:59:59,view,2701706,2053013563911439225,appliances.kitchen.refrigerators,samsung,566.27,531607492,368ddc8b-5db9-40fb-b7ff-b6582a1192c0,1
67501978,67501978,2019-12-01 08:59:59,view,1004233,2053013555631882655,electronics.smartphone,apple,1312.52,579969851,90aca71c-ed8a-4670-866a-761ebacb732d,1


In [ ]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 11 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   KTC_일시      object 
 2   이벤트 타입      object 
 3   상품ID        int64  
 4   상품 카테고리ID   int64  
 5   카테고리 코드     object 
 6   브랜드         object 
 7   가격          float64
 8   사용자ID       int64  
 9   사용자 세션ID    object 
 10  kTC_일(Day)  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 5.5+ GB


In [ ]:
# 변수(이벤트 타입) 확인: view(고객이 제품을 구경) / car(고객이 장바구니에 담음) / purchase(고객이 제품을 구매)
df["이벤트 타입"].value_counts()

,count
이벤트 타입,
view,63556110
cart,3028930
purchase,916939


## **데이터 전처리**

In [ ]:
# 데이터 재정의: 연관성 분석(장바구니 분석)을 위해 고객이 상품을 구매한 데이터만 선별

del df['Unnamed: 0']                            # 불필요한 데이터 제거
purch_df = df[df["이벤트 타입"]=='purchase']    # 고객 구매 데이터 선별
purch_df = purch_df.reset_index()               # 인덱스 초기화

print(purch_df.shape)
purch_df.tail()

(916939, 11)


,index,KTC_일시,이벤트 타입,상품ID,상품 카테고리ID,카테고리 코드,브랜드,가격,사용자ID,사용자 세션ID,kTC_일(Day)
916934,67501725,2019-12-01 08:58:14,purchase,1004874,2053013555631882655,electronics.smartphone,samsung,346.70,547804983,717566cf-ef93-4078-ba8f-169a3ac9f1a0,1
916935,67501745,2019-12-01 08:58:22,purchase,1005130,2053013555631882655,electronics.smartphone,apple,1437.02,515582054,829c20b5-696e-4a8a-8a9f-171014a3ecbe,1
916936,67501824,2019-12-01 08:58:57,purchase,1004767,2053013555631882655,electronics.smartphone,samsung,235.60,579876821,ca50e291-43f3-4ca2-9e13-20ee6b8b25f0,1
916937,67501863,2019-12-01 08:59:15,purchase,3701309,2053013565983425517,appliances.environment.vacuum,polaris,89.32,543733099,a65116f4-ac53-4a41-ad68-6606788e674c,1
916938,67501944,2019-12-01 08:59:44,purchase,26205399,2053013563693335403,NaN,NaN,179.16,553345124,58c692ff-c7a9-4e35-9ec4-58598f1940e0,1


In [ ]:
# 데이터 저장(1)
#purch_df.to_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_02_purch.csv')

---

In [ ]:
# 데이터 불러오기
purch_df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_02_purch.csv')

del purch_df['Unnamed: 0']; del purch_df['index']
print(purch_df.shape)
purch_df.tail()

(916939, 10)


,KTC_일시,이벤트 타입,상품ID,상품 카테고리ID,카테고리 코드,브랜드,가격,사용자ID,사용자 세션ID,kTC_일(Day)
916934,2019-12-01 08:58:14,purchase,1004874,2053013555631882655,electronics.smartphone,samsung,346.70,547804983,717566cf-ef93-4078-ba8f-169a3ac9f1a0,1
916935,2019-12-01 08:58:22,purchase,1005130,2053013555631882655,electronics.smartphone,apple,1437.02,515582054,829c20b5-696e-4a8a-8a9f-171014a3ecbe,1
916936,2019-12-01 08:58:57,purchase,1004767,2053013555631882655,electronics.smartphone,samsung,235.60,579876821,ca50e291-43f3-4ca2-9e13-20ee6b8b25f0,1
916937,2019-12-01 08:59:15,purchase,3701309,2053013565983425517,appliances.environment.vacuum,polaris,89.32,543733099,a65116f4-ac53-4a41-ad68-6606788e674c,1
916938,2019-12-01 08:59:44,purchase,26205399,2053013563693335403,NaN,NaN,179.16,553345124,58c692ff-c7a9-4e35-9ec4-58598f1940e0,1


In [ ]:
# 수익 극대화를 위한 마케팅을 위해 상품구매까지 이루어진 고객 중에서 구매 건 수가 100회 이상인 VIP고객 40명만 선별
VIP_Cust = list(purch_df["사용자ID"].value_counts().head(40).index)
print(len(VIP_Cust))

40


In [ ]:
# 데이터 재정의: 40명의 VIP고객 데이터만 선별
VIP_purch_df = purch_df[purch_df['사용자ID'].isin(VIP_Cust)]
VIP_purch_df = VIP_purch_df.reset_index()   # 데이터 인덱스 초기화
del VIP_purch_df["index"]                       # 데이터 변수 제거

print(VIP_purch_df.shape)
VIP_purch_df.tail()

(5749, 10)


,KTC_일시,이벤트 타입,상품ID,상품 카테고리ID,카테고리 코드,브랜드,가격,사용자ID,사용자 세션ID,kTC_일(Day)
5744,2019-12-01 08:19:05,purchase,5100566,2053013553341792533,electronics.clocks,apple,485.13,533074223,3bd08481-011e-493a-8232-6460952c95a1,1
5745,2019-12-01 08:19:51,purchase,5100566,2053013553341792533,electronics.clocks,apple,485.13,533074223,3bd08481-011e-493a-8232-6460952c95a1,1
5746,2019-12-01 08:20:53,purchase,5100566,2053013553341792533,electronics.clocks,apple,485.13,533074223,3bd08481-011e-493a-8232-6460952c95a1,1
5747,2019-12-01 08:21:41,purchase,1005107,2053013555631882655,electronics.smartphone,apple,994.88,533074223,3bd08481-011e-493a-8232-6460952c95a1,1
5748,2019-12-01 08:22:48,purchase,4804055,2053013554658804075,electronics.audio.headphone,apple,195.50,533074223,3bd08481-011e-493a-8232-6460952c95a1,1


In [ ]:
# 데이터 형태 변환: 연관성 분석을 위한 구조 변경

# 리스트 정의
Cust_rank_list = []
Cust_name_list = []
Cust_item_list = []

# 리스트 생성
for idx, Cus_id in enumerate(VIP_Cut):
  Cust_rank_list.append(idx)         # VIP고객명
  Cust_name_list.append(Cus_id)      # VIP순위
  purch_item = list(set(list(VIP_purch_df[VIP_purch_df['사용자ID']==Cus_id]['상품ID'])))
  Cust_item_list.append(purch_item)  # VIP고객의 구매 상품(중복 제거 完)

# 데이터 생성
VIP_40_Cust_df = pd.DataFrame(zip(Cust_rank_list, Cust_name_list, Cust_item_list), columns = ['고객번호', '고객ID', '구매목록'])

print(VIP_40_Cust_df.shape)
VIP_40_Cust_df.tail()

(40, 3)


,고객번호,고객ID,구매목록
35,35,561174161,"[21400576, 1002629, 1005203, 1005204, 1005205,..."
36,36,549801353,"[1005186, 26400644, 1002629, 1004934, 1002633,..."
37,37,564769842,"[1003712, 1005186, 1004648, 1005225, 1004653, ..."
38,38,514252540,"[2701705, 1801995, 1002524, 3601437, 1801631, ..."
39,39,512924342,"[1002528, 1004739, 1002532, 1005159, 1005161, ..."


In [ ]:
# 데이터 저장(2)
#VIP_40_Cust_df.to_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_02_vip_purch_list.csv')

In [ ]:
# 데이터 불러오기
#VIP_40_Cust_df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_02_vip_purch_list.csv')

#del VIP_40_Cust_df["Unnamed: 0"]

#print(VIP_40_Cust_df.shape)
#VIP_40_Cust_df.tail()

(40, 3)


,고객번호,고객ID,구매목록
35,35,561174161,"[21400576, 1002629, 1005203, 1005204, 1005205,..."
36,36,549801353,"[1005186, 26400644, 1002629, 1004934, 1002633,..."
37,37,564769842,"[1003712, 1005186, 1004648, 1005225, 1004653, ..."
38,38,514252540,"[2701705, 1801995, 1002524, 3601437, 1801631, ..."
39,39,512924342,"[1002528, 1004739, 1002532, 1005159, 1005161, ..."


In [ ]:
# 지금까지 과정을 정리하면,
# 1. 원본 고객 행동 데이터에서 '①상품을 구매 / ②구매 건수가 100건 이상 / ③상위 40명의 고객 선별'하여 데이터를 정제
# 2. 연관성 분석을 위한 데이터 형태 변환

# 이후, 본격적으로 연관성 분석~~

## **연관성 분석**

In [ ]:
# 연관성 분석을 위해 원핫 인코딩(one-hot encoding)형태로 데이터 변환

te = TransactionEncoder()                                        # transform함수를 통해, 원핫 인코딩된 numPy형태로 변환
te_ary = te.fit(Cust_item_list).transform(Cust_item_list)        # fit함수를 통해, 고객별 구매상품 항목을 고유 라벨을 갖도록 적용
final_df = pd.DataFrame(te_ary,
                  index = VIP_40_Cust_df["고객ID"],
                  columns = te.columns_)

print(final_df.shape)
final_df

(40, 640)


,1002524,1002528,1002531,1002532,1002535,1002540,1002544,1002547,1002629,1002633,...,49400040,100000121,100001025,100006662,100006963,100009409,100009643,100013289,100015156,100015658
고객ID,,,,,,,,,,,,,,,,,,,,,
564068124,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
512386086,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
549109608,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
518514099,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
549030056,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
566448225,True,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
538473314,True,False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
513230794,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
543128872,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 연관성 분석

# Apriori 알고리즘 활용
results = apriori(final_df,
                  min_support = 0.5,     # 0.5 이상 지지도 결과를 출력
                  use_colnames = True)   # True 상품 집합을 컬럼으로 설정


# 연관성 분석 결과 정리
results["length"] = results["itemsets"].apply(lambda x: len(x))  # 구매상품 개수 변수 생성
results = results.sort_values('support', ascending=False)        # support 기반 정렬
results = results.reset_index()                                  # 인덱스 초기화
del results["index"]                                             # 불필요한 변수 제거

print(results.shape)
results

(10, 3)


,support,itemsets,length
0,0.625,(1004856),1
1,0.600,(1004767),1
2,0.600,(1004833),1
3,0.600,"(1004856, 1004833)",2
4,0.575,"(1004856, 1004767)",2
5,0.550,(1005100),1
6,0.550,"(1004833, 1004767)",2
7,0.550,"(1004856, 1004833, 1004767)",3
8,0.525,(1002544),1
9,0.500,"(1004856, 1005100)",2


In [ ]:
# 연관성 분석 결과를 지지도(support)를 통해 포괄적 구매 패턴 확인
# 하지만 해당 패턴 중에서 어떤 패턴이 효율성이 좋고, 강도가 강한지 확인이 필요하다 판단

# 추가적으로 유의미한 패턴을 검증하고자 신뢰도(confidence) 및 향상도(lift)를 기반으로 분석을 진행

In [ ]:
# 성능 측정(1)
# 지지도(support)가 0.5가 넘는 항목에 대한 신뢰도(confidence)
con_results_association_rules_df = association_rules(results,
                                                     metric = "confidence",  # 성능 평가지표 설정
                                                     min_threshold = 1)      # 설정한 성능 평가지표의 설정값(기본값 0.8)
con_results_association_rules_df.columns = ['상품A', '상품B', '상품A-지지도', '상품B-지지도', '지지도', '신뢰도', '향상도', '레버리지', '컨빅션', 'zhangs_metric']
con_results_association_rules_df

,상품A,상품B,상품A-지지도,상품B-지지도,지지도,신뢰도,향상도,레버리지,컨빅션,zhangs_metric
0,(1004833),(1004856),0.60,0.625,0.60,1.0,1.6,0.22500,inf,0.937500
1,"(1004833, 1004767)",(1004856),0.55,0.625,0.55,1.0,1.6,0.20625,inf,0.833333


In [ ]:
# 성능 측정(2)
# 지지도(support)가 0.5가 넘는 항목에 대한 향상도(lift)
lif_results_association_rules_df = association_rules(results,
                                                     metric = "lift",        # 성능 평가지표 설정
                                                     min_threshold = 1)      # 설정한 성능 평가지표의 설정값(기본값 0.8)
lif_results_association_rules_df.columns = ['상품A', '상품B', '상품A-지지도', '상품B-지지도', '지지도', '신뢰도', '향상도', '레버리지', '컨빅션', 'zhangs_metric']
lif_results_association_rules_df

,상품A,상품B,상품A-지지도,상품B-지지도,지지도,신뢰도,향상도,레버리지,컨빅션,zhangs_metric
0,(1004856),(1004833),0.625,0.600,0.600,0.960000,1.600000,0.22500,10.000,1.000000
1,(1004833),(1004856),0.600,0.625,0.600,1.000000,1.600000,0.22500,inf,0.937500
2,(1004856),(1004767),0.625,0.600,0.575,0.920000,1.533333,0.20000,5.000,0.927536
3,(1004767),(1004856),0.600,0.625,0.575,0.958333,1.533333,0.20000,9.000,0.869565
4,(1004833),(1004767),0.600,0.600,0.550,0.916667,1.527778,0.19000,4.800,0.863636
5,(1004767),(1004833),0.600,0.600,0.550,0.916667,1.527778,0.19000,4.800,0.863636
6,"(1004856, 1004833)",(1004767),0.600,0.600,0.550,0.916667,1.527778,0.19000,4.800,0.863636
7,"(1004856, 1004767)",(1004833),0.575,0.600,0.550,0.956522,1.594203,0.20500,9.200,0.877005
8,"(1004833, 1004767)",(1004856),0.550,0.625,0.550,1.000000,1.600000,0.20625,inf,0.833333
9,(1004856),"(1004833, 1004767)",0.625,0.550,0.550,0.880000,1.600000,0.20625,3.750,1.000000


---

In [ ]:
# 결과해석
# 24명의 VIP 고객(전체 60%)은 1004856 상품을 구매했으며, 이들의 대부분은 1004833, 1004767, 1005100 상품을 함께 구매한 패턴을 확인했어.
# 또한 20명의 VIP 고객(전체 50%)는 1002544 상품를 구매했는데, 특히하게 해당 상품은 특별한 패턴이 없이 단독으로 많이 판매가된 것을 확인할 수 있어.

# 지지도를 0.5로 설정하여 신뢰도, 향상도, 레버리지, 컨빅션 등의 성과지표를 기반으로 해석했을 때 전반적으로 긍정적인 관계성을 확인했음.
# 보다 구체적인 해석을 신뢰도, 향상도를 중심으로 레버리지, 컨빅션을 보조적으로 확인하였음

# 1004833상품, 1004856상품, 1004767상품들은 신뢰도가 비교적 가장 높으며, 향상도, 레버리지, 컨빅션 대부분 상위 수치를 증명하였음.
# 1002544상품은 어떠한 상품과도 관련성이 없음.

# 결론적
# 1004833, 1004856, 1004767상품 간의 관련성이 전반적으로 유의미하여 묶은 패키지 형식으로 전략적 기획이 필요함.
# 1002544상품은 많은 구매패턴이 있지만 어떠한 상품과도 연관성이 없어 해당 상품을 고객들에게 자주노출시키고 관련 상품을 탐색하는 기획이 필요함.

---

In [ ]:
# 생성 데이터 설명
# 1. eCommerce_CBD_02: eC 고객 행동 데이터(원본)
# 2. eCommerce_CBD_02_purch: 고객 구매행동 데이터
# 3. eCommerce_CBD_02_vip_purch_list: vip고객의 구매 목록

---